In [1]:
import os
import sys
import math
import torch

import numpy as np
import pandas as pd
import torch.nn as nn

from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, mean_squared_error, f1_score
from sklearn.preprocessing import LabelEncoder

# Summary

The most basic Neural Matrix Factorization model possible based on Figure 3 from the original [NCF paper](https://arxiv.org/pdf/1708.05031).

This model is trained on the Movielense dataset where the objective is to predict whether a user like a movie or not (ratings above 3 mean they liked the movie).

This ultra-simplistic version uses only the user and movie IDs to create learnable embeddings, no additional user or item features are used.

In [47]:
"""
Define the general parameters that will be used.
Seeds are set for reproducibility.
Device is set to select if we will run the model on a GPU or on CPU.
All other parameters are related to the model itseld, it's optimizer and training schema.
"""


np.random.seed(42)
torch.manual_seed(42)

device = 'cpu'

learning_rate = 0.0001
weight_decay = 1e-6
batch_size = 1024
epochs = 50
hidden_layers = [32, 32]
embedding_size = 32

In [48]:
"""
Select which dataset to use.
The files in each dataset are stored differently mostly due to data size but also as its structure evolved over the years.
"""

dataset = 'ml-100k'

if dataset == 'ml-100k':
    data = pd.read_csv(f'../data/{dataset}/u.data', sep="\t", header=None)
    data.columns = ['user id', 'movie id', 'rating', 'timestamp']
elif dataset == 'ml-20m':
    data = pd.read_csv(f'../data/{dataset}/ratings.csv',on_bad_lines='skip')
else:
    print(f'{dataset} does nto exist or was incorrectly written')

In [49]:
display(data)

,user id,movie id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [50]:
"""
The embedding layers expect indices to start at 0.
Given both user and movie IDs start at 1, we apply label encoder to fix it. 
Subtracting 1 would also solve the problem however that would be problem specific and not a general solution.
"""

user_enc = LabelEncoder()
item_enc = LabelEncoder()
data['user_id_enc'] = user_enc.fit_transform(data['user id'])
data['movie_id_enc'] = item_enc.fit_transform(data['movie id'])

data['liked'] = (data['rating'] >= 3.0).astype(int)

In [51]:
data.liked.value_counts()

liked
1    82520
0    17480
Name: count, dtype: int64

In [52]:
class MLDataset(Dataset):
    """
    A PyTorch Dataset class for loading user-item interaction data for the NCF model.

    This class is designed to handle datasets containing user-item pairs and their associated ratings.
    It converts the input data into a format suitable for PyTorch's data loading utilities.

    Attributes:
    ----------
    user_item_pairs : np.ndarray
        A numpy array containing user-item pairs, where each pair consists of encoded user IDs and movie IDs.
    
    ratings : np.ndarray
        A numpy array containing the normalized ratings (scaled between 0 and 1) corresponding to each user-item pair.
    
    pair_max_ids : np.ndarray
        A numpy array containing the maximum user and movie IDs in the dataset, used for indexing purposes.

    Methods:
    -------
    __getitem__(index):
        Retrieves the user ID, movie ID, and corresponding rating for a specified index.
    
    __len__():
        Returns the total number of ratings in the dataset.

    Parameters:
    ----------
    data : pd.DataFrame
        A pandas DataFrame containing the dataset with at least the following columns 'user_id_enc', 'movie_id_enc', and 'rating'.
        - 'user_id_enc': Encoded user IDs.
        - 'movie_id_enc': Encoded movie IDs.
        - 'rating': Ratings given by users to the movies, expected to be in the range [0, 5].

    Example:
    --------
    >>> dataset = MLDataset(data)
    >>> user_id, movie_id, rating = dataset[0]
    >>> dataset_length = len(dataset)
    """
    
    def __init__(self, data):
        self.user_item_pairs = data[['user_id_enc', 'movie_id_enc']].to_numpy().astype(np.int32)
        self.ratings = data[['liked']].to_numpy().astype(np.float32)
        self.pair_max_ids = np.max(self.user_item_pairs, axis=0)

    def __getitem__(self, index):
        return self.user_item_pairs[index][0], self.user_item_pairs[index][1], self.ratings[index]

    def __len__(self):
        return self.ratings.shape[0]

In [53]:
"""
Create the train, validation and test set partitions.
Pytorch's random_split works similarly to scikit train_test_split, but allows for 2 additional important things:
 - Can partition instanciated pytorch Datasets.
 - Can split 3 ways, instead of only 2.
"""


dataset = MLDataset(data)

train_length = int(len(dataset) * 0.7)
valid_length = int(len(dataset) * 0.2)
test_length = len(dataset) - train_length - valid_length

train_dataset, valid_dataset, test_dataset = random_split(dataset, (train_length, valid_length, test_length))

train_data_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=7)
valid_data_loader = DataLoader(valid_dataset, batch_size=batch_size, num_workers=7)
test_data_loader = DataLoader(test_dataset, batch_size=1, num_workers=1)

In [54]:
class NCF(nn.Module):
    """
    Neural Collaborative Filtering (NCF) model.

    This class implements a neural network model for collaborative filtering, combining user and item embeddings 
    with a multi-layer perceptron (MLP) and a Generalized Matrix Factorization (GMF) to predict user-item interactions.
    The model uses embeddings to capture the latent factors of users and items, and then applies both a nonlinear transformation
    through the MLP, and a linear transformation through the GMF.

    Attributes:
    ----------
    user_embedding : nn.Embedding
        An embedding layer for user IDs, mapping each user to a dense vector representation of specified size.
    
    item_embedding : nn.Embedding
        An embedding layer for item IDs, mapping each item to a dense vector representation of specified size.
    
    mlp : nn.ModuleList
        A list of sequential layers forming the multi-layer perceptron, which processes the concatenated embeddings 
        to learn non-linear interactions between users and items.
    
    output : nn.Linear
        A linear layer that produces the final prediction score for user-item interactions.

    Parameters:
    ----------
    num_users : int
        The total number of unique users in the dataset.
    
    num_items : int
        The total number of unique items in the dataset.
    
    embedding_size : int, optional
        The size of the user and item embeddings (default is 32).
    
    hidden_layers : list of int, optional
        A list specifying the number of neurons in each hidden layer of the MLP (default is [32, 32]).

    Methods:
    -------
    forward(user_input, item_input):
        Defines the forward pass of the model, computing the prediction for the given user and item inputs.

    Example:
    --------
    >>> model = NCF(num_users=1000, num_items=500, embedding_size=32, hidden_layers=[64, 32])
    >>> user_input = torch.tensor([0, 1, 2])
    >>> item_input = torch.tensor([5, 6, 7])
    >>> predictions = model(user_input, item_input)
    >>> print(predictions.shape)  # Output: torch.Size([3, 1])
    """
    def __init__(self, num_users, num_items, embedding_size=32, hidden_layers=[32,32]):
        super(NCF, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_size)
        self.item_embedding = nn.Embedding(num_items, embedding_size)

        input_dim = 2 * embedding_size
        self.mlp = nn.ModuleList()
        for hidden_dim in hidden_layers:
            self.mlp.append(
                nn.Sequential(
                    nn.Linear(input_dim, hidden_dim),
                    nn.BatchNorm1d(hidden_dim),
                    nn.ReLU(),
                    nn.Dropout(p=0.2)
                )
            )
            input_dim = hidden_dim

        self.output = nn.Linear(2 * embedding_size, 1)
        
    def forward(self, user_input, item_input):
        user_embedded = self.user_embedding(user_input)
        item_embedded = self.item_embedding(item_input)
        gmf = user_embedded * item_embedded
        x = torch.cat([user_embedded, item_embedded], dim=-1)
        for mlp_layer in self.mlp:
            x = mlp_layer(x)
        x = torch.cat([gmf, x], dim=-1)
        prediction = torch.sigmoid(self.output(x))
        return prediction

In [55]:
def train(model, optimizer, data_loader, criterion, device, log_interval=10):
    """
    Train the given model using the provided data loader and optimization parameters.

    This function performs one epoch of training for the specified model, iterating over the data loader to 
    retrieve user-item interactions and their corresponding ratings. It computes the loss using the specified 
    criterion, performs backpropagation, and updates the model parameters using the optimizer.

    Parameters:
    ----------
    model : nn.Module
        The neural network model to be trained.

    optimizer : torch.optim.Optimizer
        The optimizer used.

    data_loader : DataLoader
        A PyTorch DataLoader that provides batches of user-item interactions and ratings for training.

    criterion : callable
        A loss function.

    device : torch.device
        The device on which the model and data should be processed.

    log_interval : int, optional
        The number of steps after which to log the average loss.

    Returns:
    -------
    float
        The average training loss over the epoch.

    Example:
    --------
    >>> avg_loss = train(model, optimizer, train_loader, criterion, device)
    >>> print(f"Average Training Loss: {avg_loss:.4f}")
    """
    model.train()
    total_loss = 0
    av_loss = []
    train_pbar = tqdm(data_loader, smoothing=0, mininterval=1.0)
    for i, (user, item, ratings) in enumerate(train_pbar):
        user, item, ratings = user.to(device), item.to(device), ratings.to(device)
        y = model(user, item)
        loss = criterion(y, ratings.float())
        model.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        if (i + 1) % log_interval == 0:
            loss = total_loss / log_interval
            av_loss.append(loss)
            train_pbar.set_postfix(loss=loss)
            total_loss = 0
    return np.mean(av_loss) 

In [56]:
def test(model, data_loader, device):
    """
    Evaluate the performance of the given model.

    This function sets the model to evaluation mode and computes the mean squared error over the provided dataloader.

    Parameters:
    ----------
    model : nn.Module
        The neural network model to be evaluated.

    data_loader : DataLoader
        A PyTorch DataLoader.

    device : torch.device
        The device (CPU or GPU) on which the model and data should be processed.

    Returns:
    -------
    float
        The mean squared error (MSE) between the predicted ratings and the actual ratings, scaled by a factor of 5.

    Example:
    --------
    >>> mse = test(model, test_loader, device)
    >>> print(f"Mean Squared Error: {mse:.4f}")
    """
    model.eval()
    targets, predicts = list(), list()
    with torch.no_grad():
        for user, item, ratings in tqdm(data_loader, smoothing=0, mininterval=1.0):
            user, item, ratings = user.to(device), item.to(device), ratings.to(device)
            y = model(user,item)
            predictions = (y >= 0.5)
            targets.extend(ratings.tolist())
            predicts.extend(predictions.tolist())
    return f1_score(targets, predicts, average='micro')

In [57]:
"""
Here we instantiate the model, as well as it's optimizer and loss function.
"""

model = NCF(dataset.pair_max_ids[0]+1, dataset.pair_max_ids[1]+1, embedding_size, hidden_layers).to(device)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [58]:
# Helper to calculate the number of training steps, usefull for smaller datasets like ml-100k

n_steps = math.ceil(train_length / batch_size)
n_steps

69

In [59]:
"""
Here we train and evaluate the model.
"""

metric_values = []
loss_values = []
for epoch_i in range(epochs):
    loss = train(model, optimizer, train_data_loader, criterion, device, log_interval=10)
    loss_values.append((epoch_i, loss))
    metric_train = test(model, train_data_loader, device)
    metric_valid = test(model, valid_data_loader, device)
    print('epoch:', epoch_i + 1, 'train: f1:', metric_train)
    print('epoch:', epoch_i + 1, 'validation: f1:', metric_valid)
    
    
    metric_values.append((epoch_i, metric_train, metric_valid))

100%|███████████████████████████████████████████| 20/20 [00:00<00:00, 42.31it/s]


epoch: 1 train: f1: 0.48484285714285713
epoch: 1 validation: f1: 0.48605


100%|███████████████████████████████████████████| 20/20 [00:00<00:00, 42.59it/s]


epoch: 2 train: f1: 0.5387571428571428
epoch: 2 validation: f1: 0.53935


100%|███████████████████████████████████████████| 20/20 [00:00<00:00, 42.05it/s]


...

epoch: 48 train: f1: 0.8278
epoch: 48 validation: f1: 0.82405


100%|███████████████████████████████████████████| 20/20 [00:00<00:00, 42.31it/s]


epoch: 49 train: f1: 0.828
epoch: 49 validation: f1: 0.8241


100%|███████████████████████████████████████████| 20/20 [00:00<00:00, 27.78it/s]

epoch: 50 train: f1: 0.8282285714285714
epoch: 50 validation: f1: 0.82425
